In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [2]:
olympics=pd.read_csv('athlete_events.csv')

In [3]:
#Printing column wise missing values
print(olympics.isnull().sum())

ID             0
Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     231333
dtype: int64


In [4]:
olympics['Medal'].fillna('DNW', inplace = True)

C:\Users\admin\AppData\Local\Temp\ipykernel_144\2160398606.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  olympics['Medal'].fillna('DNW', inplace = True)


In [5]:
#reading NOC dataset
noc_country = pd.read_csv('noc_regions.csv')
noc_country.drop('notes', axis = 1 , inplace = True)
noc_country.rename(columns = {'region':'Country'}, inplace = True)

In [6]:
#joining both datasets with NOC as primary key
olympics_merge = olympics.merge(noc_country,
                                left_on = 'NOC',
                                right_on = 'NOC',
                                how = 'left')

In [7]:
# Do we have NOCs that didnt have a matching country in the master?
print(olympics_merge.loc[olympics_merge['Country'].isnull(),['NOC', 'Team']].drop_duplicates())

        NOC                      Team
578     SGP                 Singapore
6267    ROT  Refugee Olympic Athletes
44376   SGP              June Climene
61080   UNK                   Unknown
64674   TUV                    Tuvalu
80986   SGP                   Rika II
108582  SGP               Singapore-2
235895  SGP               Singapore-1


In [8]:
# Replace missing Teams by the values 1. SGP - Singapore
                                    # 2. ROT - Refugee Olympic Athletes
                                    # 3. UNK - Unknown
                                    # 4. TUV - Tuvalu
#olympics_merge.loc[olympics_merge['Country'].isnull(), ['Country']] = olympics_merge['Team']

olympics_merge['Country'] = np.where(olympics_merge['NOC']=='SGP', 'Singapore', olympics_merge['Country'])
olympics_merge['Country'] = np.where(olympics_merge['NOC']=='ROT', 'Refugee Olympic Athletes', olympics_merge['Country'])
olympics_merge['Country'] = np.where(olympics_merge['NOC']=='UNK', 'Unknown', olympics_merge['Country'])
olympics_merge['Country'] = np.where(olympics_merge['NOC']=='TUV', 'Tuvalu', olympics_merge['Country'])

In [9]:
# Drop Team column and rename Country column to team column
olympics_merge.drop('Team', axis = 1, inplace = True)
olympics_merge.rename(columns = {'Country': 'Team'}, inplace = True)

In [10]:
#Checking again for mapping of NOC to team we find that each is mapped to a single value.
print(olympics_merge.loc[olympics_merge['Team'].isnull(),['NOC', 'Team']].drop_duplicates())

Empty DataFrame
Columns: [NOC, Team]
Index: []


In [11]:
#checking Null Values
print(olympics_merge.isnull().sum())

ID            0
Name          0
Sex           0
Age        9474
Height    60171
Weight    62875
NOC           0
Games         0
Year          0
Season        0
City          0
Sport         0
Event         0
Medal         0
Team          0
dtype: int64


In [12]:
# Lets take data from 1961 onwards only and for summer olympics only
olympics_complete_subset = olympics_merge.loc[(olympics_merge['Year'] > 1960) & (olympics_merge['Season'] == "Summer"), :]
print(olympics_complete_subset.head())

    ID                Name Sex   Age  Height  Weight  NOC        Games  Year  \
0    1           A Dijiang   M  24.0   180.0    80.0  CHN  1992 Summer  1992   
1    2            A Lamusi   M  23.0   170.0    60.0  CHN  2012 Summer  2012   
31  12   Jyri Tapani Aalto   M  31.0   172.0    70.0  FIN  2000 Summer  2000   
32  13  Minna Maarit Aalto   F  30.0   159.0    55.5  FIN  1996 Summer  1996   
33  13  Minna Maarit Aalto   F  34.0   159.0    55.5  FIN  2000 Summer  2000   

    Season       City       Sport                         Event Medal     Team  
0   Summer  Barcelona  Basketball   Basketball Men's Basketball   DNW    China  
1   Summer     London        Judo  Judo Men's Extra-Lightweight   DNW    China  
31  Summer     Sydney   Badminton       Badminton Men's Singles   DNW  Finland  
32  Summer    Atlanta     Sailing    Sailing Women's Windsurfer   DNW  Finland  
33  Summer     Sydney     Sailing    Sailing Women's Windsurfer   DNW  Finland  


In [13]:
# We see the row indices are not contigent anymore due to removal of rows. So,Reset row indices
olympics_complete_subset = olympics_complete_subset.reset_index()
print(olympics_complete_subset.head())

   index  ID                Name Sex   Age  Height  Weight  NOC        Games  \
0      0   1           A Dijiang   M  24.0   180.0    80.0  CHN  1992 Summer   
1      1   2            A Lamusi   M  23.0   170.0    60.0  CHN  2012 Summer   
2     31  12   Jyri Tapani Aalto   M  31.0   172.0    70.0  FIN  2000 Summer   
3     32  13  Minna Maarit Aalto   F  30.0   159.0    55.5  FIN  1996 Summer   
4     33  13  Minna Maarit Aalto   F  34.0   159.0    55.5  FIN  2000 Summer   

   Year  Season       City       Sport                         Event Medal  \
0  1992  Summer  Barcelona  Basketball   Basketball Men's Basketball   DNW   
1  2012  Summer     London        Judo  Judo Men's Extra-Lightweight   DNW   
2  2000  Summer     Sydney   Badminton       Badminton Men's Singles   DNW   
3  1996  Summer    Atlanta     Sailing    Sailing Women's Windsurfer   DNW   
4  2000  Summer     Sydney     Sailing    Sailing Women's Windsurfer   DNW   

      Team  
0    China  
1    China  
2  Finland 

In [14]:
#checking Null Values
print(olympics_complete_subset.isnull().sum())
#Null values reduced considerably by using the method of discarding tuples

index        0
ID           0
Name         0
Sex          0
Age        827
Height    7030
Weight    7294
NOC          0
Games        0
Year         0
Season       0
City         0
Sport        0
Event        0
Medal        0
Team         0
dtype: int64


In [15]:
print(olympics_complete_subset)

         index      ID                          Name Sex   Age  Height  \
0            0       1                     A Dijiang   M  24.0   180.0   
1            1       2                      A Lamusi   M  23.0   170.0   
2           31      12             Jyri Tapani Aalto   M  31.0   172.0   
3           32      13            Minna Maarit Aalto   F  30.0   159.0   
4           33      13            Minna Maarit Aalto   F  34.0   159.0   
...        ...     ...                           ...  ..   ...     ...   
158143  271106  135565      Fernando scar Zylberberg   M  27.0   168.0   
158144  271107  135566    James Francis "Jim" Zylker   M  21.0   175.0   
158145  271108  135567  Aleksandr Viktorovich Zyuzin   M  24.0   183.0   
158146  271109  135567  Aleksandr Viktorovich Zyuzin   M  28.0   183.0   
158147  271110  135568        Olga Igorevna Zyuzkova   F  33.0   171.0   

        Weight  NOC        Games  Year  Season            City       Sport  \
0         80.0  CHN  1992 Summer 

In [16]:
#Extracting unique events in a new list
listunique=olympics_complete_subset.Event.unique()
print(listunique)
print(len(listunique))

["Basketball Men's Basketball" "Judo Men's Extra-Lightweight"
 "Badminton Men's Singles" "Sailing Women's Windsurfer"
 "Athletics Men's Shot Put" "Handball Women's Handball"
 "Weightlifting Women's Super-Heavyweight"
 "Wrestling Men's Light-Heavyweight, Greco-Roman"
 "Water Polo Men's Water Polo" 'Sailing Mixed Three Person Keelboat'
 "Hockey Women's Hockey" "Rowing Men's Lightweight Double Sculls"
 "Sailing Men's Two Person Dinghy" "Athletics Men's 1,500 metres"
 "Swimming Men's 100 metres Butterfly"
 "Swimming Men's 200 metres Butterfly"
 "Swimming Men's 4 x 100 metres Medley Relay" "Football Women's Football"
 'Equestrianism Mixed Jumping, Individual'
 "Boxing Men's Light-Welterweight" "Athletics Women's Javelin Throw"
 "Wrestling Men's Heavyweight, Freestyle"
 "Gymnastics Men's Individual All-Around"
 "Gymnastics Men's Floor Exercise" "Gymnastics Men's Parallel Bars"
 "Gymnastics Men's Horizontal Bar" "Gymnastics Men's Rings"
 "Gymnastics Men's Pommelled Horse" "Athletics Men's Pol

In [17]:
tempagemean=[]
tempweightmean=[]
tempheightmean=[]

#looping to the event list and creating new lists for means of age,weight and height for unique events.
for i in listunique:
    temporary=olympics_complete_subset.loc[(olympics_complete_subset['Event'] == i) & (olympics_complete_subset['Age'] !='NaN' ), :]
    tempagemean.append(temporary.Age.mean(axis = 0, skipna = True))
    tempweightmean.append(temporary.Weight.mean(axis=0,skipna = True))
    tempheightmean.append(temporary.Height.mean(axis = 0, skipna = True))

In [18]:
#looping for each event(since there are 372 events)
for i in range(0,372):
    #extracting rows with a particular event into another dataframe
    df1 = olympics_complete_subset[olympics_complete_subset['Event'].str.contains(listunique[i]) ]
    #filling null values for age in the new dataframe to the mean age for that event
    df1.Age.fillna(tempagemean[i],inplace=True)
    # filling null values for weight in the new dataframe to the mean weight for that event
    df1.Weight.fillna(tempweightmean[i], inplace=True)
    # filling null values for height in the new dataframe to the mean height for that event
    df1.Height.fillna(tempheightmean[i], inplace=True)
    #Dropping columns for that particular event in the original dataframe
    olympics_complete_subset.drop(olympics_complete_subset[olympics_complete_subset['Event'].str.contains(listunique[i])].index, inplace = True)
    #concatinating the altered rows in new dataframe to the old dataframe
    olympics_complete_subset = pd.concat([olympics_complete_subset, df1], axis=0)

C:\Users\admin\AppData\Local\Temp\ipykernel_144\653981728.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1.Age.fillna(tempagemean[i],inplace=True)
C:\Users\admin\AppData\Local\Temp\ipykernel_144\653981728.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.Age.fillna(tempagemean[i],inplace=True)
C:\Users\admin\AppData\Local\Temp\ipy

In [19]:
#Print statements to check for null values
print(olympics_complete_subset.head())
print(olympics_complete_subset.isnull().sum())
print(olympics_complete_subset)

     index   ID                                Name Sex   Age  Height  Weight  \
0        0    1                           A Dijiang   M  24.0   180.0    80.0   
135    264  136                    Alessandro Abbio   M  29.0   195.0    85.0   
187    346  192  Ahmed El-Sayed Abdel Hamid Mobarak   M  25.0   189.0    85.0   
191    359  199                   Amir Abdel Meguid   M  23.0   178.0    67.0   
192    360  199                   Amir Abdel Meguid   M  27.0   178.0    67.0   

     NOC        Games  Year  Season         City       Sport  \
0    CHN  1992 Summer  1992  Summer    Barcelona  Basketball   
135  ITA  2000 Summer  2000  Summer       Sydney  Basketball   
187  EGY  1972 Summer  1972  Summer       Munich  Basketball   
191  EGY  1984 Summer  1984  Summer  Los Angeles  Basketball   
192  EGY  1988 Summer  1988  Summer        Seoul  Basketball   

                           Event Medal   Team  
0    Basketball Men's Basketball   DNW  China  
135  Basketball Men's Basketball

In [20]:
#giving numeric value to gender column 1 for male and 0 for female
olympics_complete_subset['Sex'] = np.where(olympics_complete_subset['Sex']=='M', 1 , 0)

In [21]:
#Creating a column that captures whether or not a medal was won! It would be 1 if Medal column says Gold, Silver or Bronze and 0 otherwise.
#Converting Medal column to numeric form
olympics_complete_subset['Medal_Won'] = np.where(olympics_complete_subset.loc[:,'Medal'] == 'DNW', 0 , olympics_complete_subset['Medal'])
olympics_complete_subset['Medal_Won'] = np.where(olympics_complete_subset.loc[:,'Medal_Won'] == 'Bronze', 1 , olympics_complete_subset['Medal_Won'])
olympics_complete_subset['Medal_Won'] = np.where(olympics_complete_subset.loc[:,'Medal_Won'] == 'Silver', 2 , olympics_complete_subset['Medal_Won'])
olympics_complete_subset['Medal_Won'] = np.where(olympics_complete_subset.loc[:,'Medal_Won'] == 'Gold', 3 , olympics_complete_subset['Medal_Won'])
print(olympics_complete_subset['Medal_Won'].unique())

[0 3 1 2]


In [22]:
#Dropping unnecessary columns from dataFrame
olympics_complete_subset.drop(['index'] , axis=1, inplace=True)
olympics_complete_subset.drop(['Name'] , axis=1, inplace=True)
olympics_complete_subset.drop(['NOC'] , axis=1, inplace=True)
olympics_complete_subset.drop(['Season'] , axis=1, inplace=True)
olympics_complete_subset.drop(['Games'] , axis=1, inplace=True)
olympics_complete_subset.drop(['City'] , axis=1, inplace=True)
olympics_complete_subset.drop(['Year'] , axis=1, inplace=True)
olympics_complete_subset.drop(['Sport'] , axis=1, inplace=True)
olympics_complete_subset.drop(['Medal'] , axis=1, inplace=True)
olympics_complete_subset.drop(['ID'] , axis=1, inplace=True)

In [23]:
# We see the row indices are not contigent anymore due to removal of rows. So,Reset row indices
olympics_complete_subset = olympics_complete_subset.reset_index()

#converting Team and Event to numerical form

In [24]:
#Creating object of preprocessing LabelEncoder
le = preprocessing.LabelEncoder()
#Creating a new column of encoded Team and Encoded Event in numerical form in original dataset
olympics_complete_subset['Team_encode'] = le.fit_transform(olympics_complete_subset['Team'])
olympics_complete_subset['Event_encode'] = le.fit_transform(olympics_complete_subset['Event'])

In [25]:
#storing Team names and corresponding  encoded numerical values into new csv file after sorting them according to Team name
TeamKeys=olympics_complete_subset[['Team', 'Team_encode']].copy()
TeamKeys.drop_duplicates(subset ="Team", inplace = True)
TeamKeys.sort_values("Team", axis = 0, ascending = True, inplace = True, na_position ='last')
TeamKeys.to_csv('keysToTeam.csv')
print(TeamKeys.head())

                 Team  Team_encode
24011     Afghanistan            0
3106          Albania            1
2272          Algeria            2
3154   American Samoa            3
2351          Andorra            4


In [26]:
#storing Event names and corresponding  encoded numerical values into new csv file after sorting them according to Event name
EventKeys=olympics_complete_subset[['Event' , 'Event_encode']].copy()
EventKeys.drop_duplicates(subset ="Event", inplace = True)
EventKeys.sort_values("Event", axis = 0, ascending = True, inplace = True, na_position ='last')
EventKeys.to_csv('keysToEvent.csv')
print(EventKeys.head())

                               Event  Event_encode
117345      Archery Men's Individual             0
118080            Archery Men's Team             1
62659     Archery Women's Individual             2
130135          Archery Women's Team             3
10085   Athletics Men's 1,500 metres             4


In [27]:
#Creating a new dataframe to store keys to Sex and store it in new csv file
SexKeys = pd.DataFrame( {'Sex': ['Male','Female'], 'Sex_encode': [1,0] }, index =[1,2] )
SexKeys.to_csv('keysToSex.csv')
print(SexKeys)

      Sex  Sex_encode
1    Male           1
2  Female           0


In [28]:
'''after mapping the key value pairs, dropping the team and Event columns from final dataset and changing
names of Team_encode to Team and Event_encode to Event in the original dataframe'''
olympics_complete_subset.drop(['Team'] , axis=1, inplace=True)
olympics_complete_subset.drop(['Event'] , axis=1, inplace=True)
olympics_complete_subset.rename(columns = {'Team_encode': 'Team'}, inplace = True)
olympics_complete_subset.rename(columns = {'Event_encode': 'Event'}, inplace = True)

In [29]:
olympics_complete_subset.drop(['index'] , axis=1, inplace=True)
#Converting the final dataset into a new csv file
olympics_complete_subset.to_csv('olympics_final_final.csv');
print(olympics_complete_subset.head())

   Sex   Age  Height  Weight Medal_Won  Team  Event
0    1  24.0   180.0    80.0         0    39     61
1    1  29.0   195.0    85.0         0    87     61
2    1  25.0   189.0    85.0         0    55     61
3    1  23.0   178.0    67.0         0    55     61
4    1  27.0   178.0    67.0         0    55     61
